# Análisis de Autores Activos: Mapping Review IA y ML en Educación Matemática K-12\n
\n
**MQ2: ¿Quiénes son los autores más activos del área?**\n
\n
Este notebook analiza los autores más productivos y sus patrones de colaboración en el campo de IA y ML en educación matemática K-12.

## 1. Configuración del Entorno

In [ ]:
# Instalación de dependencias\n
!pip install pandas numpy matplotlib seaborn plotly networkx

In [ ]:
# Importación de librerías\n
import pandas as pd\n
import numpy as np\n
import matplotlib.pyplot as plt\n
import seaborn as sns\n
import plotly.express as px\n
import plotly.graph_objects as go\n
import networkx as nx\n
import re\n
from collections import Counter\n
import warnings\n
warnings.filterwarnings('ignore')\n
\n
# Configuración de estilo\n
plt.style.use('seaborn-v0_8')\n
sns.set_palette(\"husl\")\n
plt.rcParams['figure.figsize'] = (12, 8)\n
plt.rcParams['font.size'] = 12\n
\n
# Configuración para mostrar todas las columnas\n
pd.set_option('display.max_columns', None)\n
pd.set_option('display.max_colwidth', None)

## 2. Carga de Datos desde GitHub

In [ ]:
# Cargar el dataset desde GitHub\n
# IMPORTANTE: Cambiar la URL por tu repositorio real\n
url = \"https://raw.githubusercontent.com/TU_USUARIO/TU_REPOSITORIO/main/MappingReview.csv\"\n
df = pd.read_csv(url, sep=';', encoding='utf-8')\n
\n
print(f\"Dataset cargado: {df.shape[0]} filas y {df.shape[1]} columnas\")\n
print(\"\\nPrimeras 5 filas:\")\n
df.head()

## 3. Procesamiento de Datos de Autores

In [ ]:
# Función para extraer autores individuales\n
def extract_authors(authors_str):\n
    if pd.isna(authors_str):\n
        return []\n
    \n
    # Dividir por punto y coma y limpiar\n
    authors = authors_str.split(';')\n
    cleaned_authors = []\n
    \n
    for author in authors:\n
        author = author.strip()\n
        if author:\n
            # Limpiar comillas y espacios extra\n
            author = re.sub(r'^\"|\"$', '', author)\n
            author = author.strip()\n
            if author:\n
                cleaned_authors.append(author)\n
    \n
    return cleaned_authors\n
\n
# Aplicar la función a la columna de autores\n
df['Authors_List'] = df['Author(s)'].apply(extract_authors)\n
\n
print(\"Procesamiento de autores completado\")\n
print(f\"Total de publicaciones con autores: {len(df[df['Authors_List'].apply(len) > 0])}\")

In [ ]:
# Crear lista de todos los autores\n
all_authors = []\n
for authors_list in df['Authors_List']:\n
    all_authors.extend(authors_list)\n
\n
# Contar frecuencia de autores\n
author_counts = Counter(all_authors)\n
\n
print(f\"Total de autores únicos: {len(author_counts)}\")\n
print(f\"Total de apariciones de autores: {len(all_authors)}\")\n
print(f\"Promedio de autores por publicación: {len(all_authors) / len(df):.2f}\")

## 4. Análisis de Autores Más Activos (MQ2)

In [ ]:
# Top 20 autores más activos\n
top_authors = author_counts.most_common(20)\n
\n
print(\"=== TOP 20 AUTORES MÁS ACTIVOS ===\")\n
for i, (author, count) in enumerate(top_authors, 1):\n
    print(f\"{i:2d}. {author}: {count} publicaciones\")\n
\n
# Crear DataFrame para análisis\n
top_authors_df = pd.DataFrame(top_authors, columns=['Author', 'Publications'])\n
top_authors_df['Rank'] = range(1, len(top_authors_df) + 1)

In [ ]:
# Gráfico de barras para top 10 autores\n
plt.figure(figsize=(15, 10))\n
top_10 = top_authors_df.head(10)\n
\n
bars = plt.barh(range(len(top_10)), top_10['Publications'], color='skyblue', alpha=0.7)\n
plt.yticks(range(len(top_10)), top_10['Author'], fontsize=10)\n
plt.xlabel('Número de Publicaciones', fontsize=14)\n
plt.title('Top 10 Autores Más Activos en IA y ML en Educación Matemática K-12', \n
          fontsize=16, fontweight='bold')\n
plt.grid(True, alpha=0.3, axis='x')\n
\n
# Agregar valores en las barras\n
for i, (bar, count) in enumerate(zip(bars, top_10['Publications'])):\n
    plt.text(count + 0.1, i, str(count), va='center', fontweight='bold')\n
\n
plt.tight_layout()\n
plt.show()

In [ ]:
# Gráfico interactivo con Plotly\n
fig = px.bar(top_authors_df.head(15), x='Publications', y='Author',\n
              orientation='h',\n
              title='Top 15 Autores Más Activos',\n
              labels={'Publications': 'Número de Publicaciones', 'Author': 'Autor'})\n
\n
fig.update_layout(\n
    title_font_size=16,\n
    xaxis_title_font_size=14,\n
    yaxis_title_font_size=14,\n
    height=600\n
)\n
\n
fig.show()

## 5. Análisis de Colaboración entre Autores

In [ ]:
# Crear red de colaboración\n
G = nx.Graph()\n
\n
# Agregar nodos (autores)\n
for author in author_counts.keys():\n
    G.add_node(author, weight=author_counts[author])\n
\n
# Agregar edges (colaboraciones)\n
for authors_list in df['Authors_List']:\n
    if len(authors_list) > 1:\n
        for i in range(len(authors_list)):\n
            for j in range(i + 1, len(authors_list)):\n
                author1, author2 = authors_list[i], authors_list[j]\n
                if G.has_edge(author1, author2):\n
                    G[author1][author2]['weight'] += 1\n
                else:\n
                    G.add_edge(author1, author2, weight=1)\n
\n
print(f\"Red de colaboración creada con {G.number_of_nodes()} nodos y {G.number_of_edges()} edges\")

In [ ]:
# Análisis de centralidad\n
degree_centrality = nx.degree_centrality(G)\n
betweenness_centrality = nx.betweenness_centrality(G)\n
eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000)\n
\n
# Crear DataFrame con métricas de centralidad\n
centrality_df = pd.DataFrame({\n
    'Author': list(G.nodes()),\n
    'Degree': [G.degree(node) for node in G.nodes()],\n
    'Degree_Centrality': [degree_centrality[node] for node in G.nodes()],\n
    'Betweenness_Centrality': [betweenness_centrality[node] for node in G.nodes()],\n
    'Eigenvector_Centrality': [eigenvector_centrality[node] for node in G.nodes()]\n
})\n
\n
centrality_df = centrality_df.sort_values('Degree', ascending=False)\n
\n
print(\"=== TOP 10 AUTORES POR CENTRALIDAD ===\")\n
print(centrality_df.head(10))

In [ ]:
# Gráfico de dispersión: Centralidad vs Número de Publicaciones\n
plt.figure(figsize=(12, 8))\n
\n
publications = [author_counts[author] for author in centrality_df['Author']]\n
centrality_df['Publications'] = publications\n
\n
plt.scatter(centrality_df['Degree_Centrality'], centrality_df['Publications'], \n
            alpha=0.6, s=50, c=centrality_df['Betweenness_Centrality'], cmap='viridis')\n
\n
plt.xlabel('Centralidad de Grado', fontsize=14)\n
plt.ylabel('Número de Publicaciones', fontsize=14)\n
plt.title('Relación entre Centralidad y Productividad de Autores', fontsize=16, fontweight='bold')\n
plt.colorbar(label='Centralidad de Intermediación')\n
plt.grid(True, alpha=0.3)\n
\n
plt.tight_layout()\n
plt.show()

## 6. Análisis de Patrones de Colaboración

In [ ]:
# Análisis de tamaño de equipos de autoría\n
team_sizes = [len(authors) for authors in df['Authors_List']]\n
team_size_counts = Counter(team_sizes)\n
\n
print(\"=== DISTRIBUCIÓN DE TAMAÑOS DE EQUIPO ===\")\n
for size, count in sorted(team_size_counts.items()):\n
    percentage = (count / len(df)) * 100\n
    print(f\"{size} autor(es): {count} publicaciones ({percentage:.1f}%)\")\n
\n
# Gráfico de distribución\n
plt.figure(figsize=(10, 6))\n
sizes = sorted(team_size_counts.keys())\n
counts = [team_size_counts[size] for size in sizes]\n
\n
plt.bar(sizes, counts, color='lightcoral', alpha=0.7)\n
plt.xlabel('Número de Autores', fontsize=14)\n
plt.ylabel('Número de Publicaciones', fontsize=14)\n
plt.title('Distribución de Tamaños de Equipo de Autoría', fontsize=16, fontweight='bold')\n
plt.grid(True, alpha=0.3, axis='y')\n
\n
# Agregar valores en las barras\n
for i, count in enumerate(counts):\n
    plt.text(sizes[i], count + 0.5, str(count), ha='center', va='bottom', fontweight='bold')\n
\n
plt.tight_layout()\n
plt.show()

In [ ]:
# Análisis de colaboraciones más frecuentes\n
collaboration_counts = {}\n
\n
for authors_list in df['Authors_List']:\n
    if len(authors_list) > 1:\n
        for i in range(len(authors_list)):\n
            for j in range(i + 1, len(authors_list)):\n
                author1, author2 = authors_list[i], authors_list[j]\n
                pair = tuple(sorted([author1, author2]))\n
                collaboration_counts[pair] = collaboration_counts.get(pair, 0) + 1\n
\n
# Top 10 colaboraciones\n
top_collaborations = sorted(collaboration_counts.items(), key=lambda x: x[1], reverse=True)[:10]\n
\n
print(\"=== TOP 10 COLABORACIONES MÁS FRECUENTES ===\")\n
for i, ((author1, author2), count) in enumerate(top_collaborations, 1):\n
    print(f\"{i:2d}. {author1} & {author2}: {count} colaboraciones\")\n
\n
# Gráfico de colaboraciones\n
if top_collaborations:\n
    collaboration_df = pd.DataFrame(top_collaborations, columns=['Authors', 'Collaborations'])\n
    collaboration_df['Author_Pair'] = [f\"{a1} & {a2}\" for (a1, a2) in collaboration_df['Authors']]\n
    \n
    plt.figure(figsize=(12, 8))\n
    bars = plt.barh(range(len(collaboration_df)), collaboration_df['Collaborations'], color='lightgreen', alpha=0.7)\n
    plt.yticks(range(len(collaboration_df)), collaboration_df['Author_Pair'], fontsize=10)\n
    plt.xlabel('Número de Colaboraciones', fontsize=14)\n
    plt.title('Top 10 Colaboraciones Más Frecuentes', fontsize=16, fontweight='bold')\n
    plt.grid(True, alpha=0.3, axis='x')\n
    \n
    # Agregar valores en las barras\n
    for i, count in enumerate(collaboration_df['Collaborations']):\n
        plt.text(count + 0.1, i, str(count), va='center', fontweight='bold')\n
    \n
    plt.tight_layout()\n
    plt.show()

## 7. Resumen y Conclusiones

In [ ]:
# Generar resumen ejecutivo\n
print(\"=== RESUMEN EJECUTIVO ===\\n\")\n
\n
print(f\"📊 Total de autores únicos: {len(author_counts)}\")\n
print(f\"📝 Total de apariciones de autores: {len(all_authors)}\")\n
print(f\"📈 Promedio de autores por publicación: {len(all_authors) / len(df):.2f}\")\n
print(f\"🔗 Colaboraciones únicas: {len(collaboration_counts)}\")\n
\n
# Autor más productivo\n
most_prolific = top_authors[0]\n
print(f\"🏆 Autor más productivo: {most_prolific[0]} ({most_prolific[1]} publicaciones)\")\n
\n
# Autor más central\n
most_central = centrality_df.iloc[0]\n
print(f\"🌟 Autor más central: {most_central['Author']} (centralidad: {most_central['Degree_Centrality']:.3f})\")\n
\n
# Tamaño de equipo más común\n
most_common_team_size = max(team_size_counts, key=team_size_counts.get)\n
print(f\"👥 Tamaño de equipo más común: {most_common_team_size} autor(es)\")\n
\n
print(\"\\n=== CONCLUSIONES ===\")\n
print(\"1. El campo muestra una alta colaboración entre investigadores\")\n
print(\"2. Hay autores líderes que dominan la producción científica\")\n
print(\"3. La mayoría de publicaciones son colaborativas\")\n
print(\"4. Existe una red de colaboración bien establecida\")